In [0]:
from cni_connectors import adls_gen1_connector as adls_conn
from pyspark.sql.functions import udf, from_utc_timestamp, current_timestamp, lit, input_file_name, monotonically_increasing_id,substring
import pyspark.sql.functions as f

import crawler.functions as cf
import json
import re

In [0]:
var_adls_uri = adls_conn.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev

#IMPLEMENTATION

This cell is for implementing widgets.get and json convertion
Provided that it is still not implemented, i'll mock it up by setting the necessary stuff for the table I'm working with.

Remember that when parsing any json, we must handle any possibility of strange char, escapes ans whatever comes dirt from Data Factory!

In [0]:
var_file = json.loads(re.sub("\'", '\"', dbutils.widgets.get("file")))
var_dls = json.loads(re.sub("\'", '\"', dbutils.widgets.get("dls")))
var_adf = json.loads(re.sub("\'", '\"', dbutils.widgets.get("adf")))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4198474377737711> in <module> 
 ----> 1 var_file = json . loads ( re . sub ( "\'" , '\"' , dbutils . widgets . get ( "file" ) ) ) 
 2 var_dls = json . loads ( re . sub ( "\'" , '\"' , dbutils . widgets . get ( "dls" ) ) ) 
 3 var_adf = json . loads ( re . sub ( "\'" , '\"' , dbutils . widgets . get ( "adf" ) ) ) 

 /databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get (self, name) 
 40 : return : Current value of the widget or default value
 41 """
 ---> 42 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 43 
 44 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o296.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named file is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
var_file = {
  'namespace': 'me',
  'file_folder': 'comex_stat/expo_ncm',
  'extension': 'csv',
  'column_delimiter': ';',
  'encoding': 'UTF-8',
  'null_value': ''
}

var_adf = {
  "adf_factory_name": "cnibigdatafactory",
  "adf_pipeline_name": "org_raw_comex_stat_expo_ncm",
  "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
  "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
  "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
  "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
  "adf_trigger_type": "PipelineActivity"
}

var_dls = {"folders":{"landing":"/tmp/dev/uld","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","archive":"/tmp/dev/ach"}, "systems":{"raw":"usr"}}

In [0]:
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']
sys = var_dls['systems']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[21]: '/tmp/dev/uld/me/comex_stat/expo_ncm/'

In [0]:
var_sink = "{adl_path}{raw}/usr/{namespace}/{file_folder}".format(adl_path=var_adls_uri, raw=raw, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_sink

Out[22]: '[REDACTED]/tmp/dev/raw/usr/me/comex_stat/expo_ncm'

In [0]:
import crawler.functions as cf

if not cf.directory_exists(dbutils, var_source):
  dbutils.notebook.exit('Path "%s" not exist or is empty' % var_source)

In [0]:
df = spark.read.csv(path=var_adls_uri + var_source, sep=var_file['column_delimiter'], encoding='UTF-8', header=True,
                    nullValue='', mode="FAILFAST", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True,
                   inferSchema=False)

In [0]:
# Files are tricky considering columns. Let's lower them all.
for c in df.columns:
  df = df.withColumnRenamed(c, c.lower())

In [0]:
df = df\
.withColumnRenamed('co_ano', 'dt_ano')\
.withColumnRenamed('co_mes', 'dt_mes')\
.withColumnRenamed('co_ncm', 'cd_ncm')\
.withColumnRenamed('co_unid', 'cd_unid')\
.withColumnRenamed('co_pais', 'cd_pais')\
.withColumnRenamed('sg_uf_ncm', 'nm_uf_ncm')\
.withColumnRenamed('co_via', 'cd_via')\
.withColumnRenamed('co_urf', 'cd_urf')\
.withColumnRenamed('qt_estat', 'vl_qt_estat')\
.withColumnRenamed('kg_liquido', 'vl_kg_liquido')\
.withColumnRenamed('vl_fob', 'vl_fob')

In [0]:
dt_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

In [0]:
df = cf.append_control_columns(df, dt_insertion_raw)

In [0]:
adl_file_time = cf.list_adl_files(spark, dbutils, var_source)
df = df.join(adl_file_time, on='nm_arq_in', how='inner')

In [0]:
# RAW
df.write.save(path=var_sink, format="parquet", mode="overwrite")